# Homework 10
## Unit 7
#### MATH620
#### Joshua Dunne

# Question 1
## Find the singular value decomposition of the matrix
$$\mathbf{A}=\begin{bmatrix}1 & 1 \\ 1 & 0 \\ 0 & 1\end{bmatrix}$$

I'm going to borrow substantially from something I wrote for the first question here, we want the SVD, but, we want to
show the steps.

In [7]:
import numpy as np

A = np.array([
    [1,1],
    [1,0],
    [0,1]
])

U, Sigma, Vt =  np.linalg.svd(A)
print(U)
print(np.diag(Sigma))
print(Vt)


[[-8.16496581e-01  1.85577521e-16 -5.77350269e-01]
 [-4.08248290e-01 -7.07106781e-01  5.77350269e-01]
 [-4.08248290e-01  7.07106781e-01  5.77350269e-01]]
[[1.73205081 0.        ]
 [0.         1.        ]]
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]


Gives us the long and short, but, you were interested in the process, so, we can step through the code I wrote to get the SVD. Hopefully that should prove sufficient.